We want to be able to generate synthetic timeseries data to test and compare our algorithms.
In order to have some control over the trends in the timeseries, we use the simple package timeseries generator in this
repository.

In [ ]:
from timeseries_generator import LinearTrend, Generator, WhiteNoise, RandomFeatureFactor
import pandas as pd

%matplotlib inline

We can now start by setting up a linear trend.
Trends are collected in a generator, which takes any dependent variables as features and the trends you have added to
compose the time-series dataframe.

In [ ]:
lt = LinearTrend(coef=2.0, offset=1., col_name="my_linear_trend")

g: Generator = Generator(factors={lt}, features=None, date_range=pd.date_range(start="01-01-2020", end="01-20-2020"))
g.generate()
g.plot()

We can update our package by adding some white noise to the generator. The white noise adds noise with a standard
deviation as a fraction of the total value to our timeseries.

In [ ]:
wn = WhiteNoise(stdev_factor=0.05)
g.update_factor(wn)
g.generate()
g.plot()

We can add dependent feature columns to our timeseries. In this case, we introduce a random factor to these features.
This factor will be dependent on a feature that we introduce to the generator.

In [ ]:
rff = RandomFeatureFactor(
    feature="my_feature",
    feature_values=["feature1", "feature2"],
    min_factor_value=1,
    max_factor_value=10
)
g.update_factor(rff)
g.features = {"my_feature": ["feature1", "feature2"]}
df: pd.DataFrame = g.generate()
print(df.head(5))

The resulting dataframe is a dataframe with all the features and factors influencing the features. We can plot the final value per product.

In [ ]:
df_plot=df.set_index('date')
df_plot[["my_feature", "value"]].pivot(columns=["my_feature"], values="value").plot()

We can also add different linear trends for different feature values.

In [ ]:
lt2 = LinearTrend(feature="my_feature", feature_values={
    "feature1": {"coef": 1., "offset": 1.},
    "feature2": {"coef": 0.05, "offset": 1.}
})

g.update_factor(lt2)
df = g.generate()

df_plot=df.set_index('date')
df_plot[["my_feature", "value"]].pivot(columns=["my_feature"], values="value").plot()